# Tải các package cần thiết

In [2]:
!pip install -q langchain langchain_groq langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


# 1.Combine các thành phần thông qua '|'


In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain_groq import ChatGroq
from google.colab import userdata

prompt = ChatPromptTemplate.from_messages(
    [
      ('system', 'Bạn là một trợ lý ảo hữu ích và am hiểu về lịch sử. Lưu ý: Chỉ được trả lời câu hỏi bằng tiếng việt'),
      ('human', '{question}')
    ]
)

llm = ChatGroq(
    model = 'llama3-70b-8192',
    api_key = userdata.get('GROQ_API_KEY')
)

chain = prompt | llm | StrOutputParser()

response = chain.stream({'question': "Ngày giải phóng Miền Nam Việt Nam là ngày nào?"})
for chuck in response:
    print(chuck, end="", flush=True)

Ngày giải phóng Miền Nam Việt Nam là ngày 30 tháng 4 năm 1975. Đây là ngày quân đội nhân dân Việt Nam và các lực lượng vũ trang giải phóng Miền Nam đã tiêu diệt hoàn toàn chế độ Việt Nam Cộng Hòa, thống nhất đất nước dưới chính quyền Cộng hòa Xã hội Chủ nghĩa Việt Nam.

In [15]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt, output_parser = StrOutputParser())
response = chain.run({'question': "Ngày giải phóng Miền Nam Việt Nam là ngày nào?"})
print(response)

Ngày giải phóng Miền Nam Việt Nam là ngày 30 tháng 4 năm 1975. Đây là ngày quân đội nhân dân Việt Nam đã chiến thắng và giải phóng hoàn toàn Miền Nam khỏi chế độ Việt Nam Cộng hòa, thống nhất đất nước dưới một chính phủ.


# 3. Sử dụng ConversationChain

In [18]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate


template = '''
Bạn là một chatbot hữu ích. Bạn là người am hiểu về lịch sử. Lưu ý: Bạn chỉ được sử dụng Tiếng Việt để trả lời
Current conversation:
{history}
Human: {input}
AI assistant:
'''

prompts = PromptTemplate(
    input_variables = ['history', 'input'],
    template = template
    )
conversation = ConversationChain(prompt=prompts,
                                 llm=llm,
                                 memory=ConversationBufferMemory(ai_prefix="AI Assistant"),
                                 verbose=True)

conversation.run(input = "Thế chiến thứ 2 diễn ra trong hoàn cảnh nào ?")





> Entering new ConversationChain chain...
Prompt after formatting:

Bạn là một chatbot hữu ích. Bạn là người am hiểu về lịch sử. Lưu ý: Bạn chỉ được sử dụng Tiếng Việt để trả lời
Current conversation:

Human: Thế chiến thứ 2 diễn ra trong hoàn cảnh nào ?
AI assistant:


> Finished chain.


'Thế chiến thứ 2 diễn ra trong hoàn cảnh của sự hỗn loạn và bất ổn chính trị trên toàn cầu sau Chiến tranh thế giới thứ nhất. Sau chiến tranh, các nước thắng trận như Anh, Pháp, và Hoa Kỳ đã áp đặt các điều khoản hòa bình khắc nghiệt lên Đức, bao gồm trả nợ chiến tranh và cắt giảm quân đội. Điều này đã dẫn đến sự sụp đổ kinh tế và bất ổn chính trị tại Đức.\n\nTrong khi đó, tại Liên Xô, Vladimir Lenin và sau đó là Joseph Stalin đã thiết lập chế độ cộng sản, và bắt đầu mở rộng ảnh hưởng của mình ra toàn cầu. Tại Ý, Benito Mussolini đã thành lập chế độ phát xít, và tại Đức, Adolf Hitler đã lên nắm quyền và bắt đầu thực hiện các chính sách膨n bành quân sự và chủ nghĩa bài Do Thái.\n\nTrong hoàn cảnh như vậy, các nước châu Âu đã bị chia rẽ và không thể thống nhất lại để chống lại sự bành trướng của Đức, Ý, và Nhật Bản. Cuối cùng, vào ngày 1 tháng 9 năm 1939, Đức dưới sự lãnh đạo của Adolf Hitler đã xâm lược Ba Lan, và Anh, Pháp đã tuyên chiến với Đức, đánh dấu sự bắt đầu của Thế chiến thứ 2.

# 4. Thêm 1 số thành phần vào chain

## 4.1 Thêm BaseModel từ Pydantic

In [ ]:
  from langchain.pydantic_v1 import BaseModel, Field
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from typing import Optional


class Person(BaseModel):
    name: str = Field(..., description="The person's name")
    age: int = Field(...,  description="The person's age")
    fav_food: Optional[str]  = Field(None, description="The person's favorite food")

llm = ChatGroq(
    model="llama3-70b-8192",
    api_key= userdata.get('GROQ_API_KEY')
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Bạn là 1 chat bot hữu ích. Lưu ý: bạn chỉ được sử dụng tiếng việt để trả lời"),
        ("user", "{input}")
    ]
)

chain = create_structured_output_runnable(Person, llm=llm, prompt=prompt)
chain.invoke({"input": "Sally 20 tuổi và cô ấy rất thích ăn bánh mì"})

## 4.2 Chỉ định các thuộc tính dự kiến


In [22]:
from langchain.chains import create_tagging_chain, create_extraction_chain_pydantic
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}


llm = ChatGroq(
    model="llama3-70b-8192",
    api_key= userdata.get("GROQ_API_KEY")
)

chain = create_tagging_chain(schema, llm)
chain.invoke("Điều này thật bất ngờ !!")

<ipython-input-22-5b3c368b3e19>:19: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` that is available on ChatModels capable of tool calling. See API reference for this function for replacement: <https://api.python.langchain.com/en/latest/chains/langchain.chains.openai_functions.tagging.create_tagging_chain.html> You can read more about `with_structured_output` here: <https://python.langchain.com/docs/how_to/structured_output/>. If you notice other issues, please provide feedback here: <https://github.com/langchain-ai/langchain/discussions/18154>
  chain = create_tagging_chain(schema, llm)


{'input': 'Điều này thật bất ngờ !!',
 'text': {'aggressiveness': 0, 'language': 'vi', 'sentiment': 'neutral'}}

## 4.3 Gọi liên tục LLM

In [23]:
synopsis_prompt = PromptTemplate.from_template(
    """Bạn là một nhà viết kịch. Với tựa đề của vở kịch, nhiệm vụ của bạn là viết tóm tắt cho tựa đề đó: {title}
    Lưu ý: Bạn chỉ được sử dụng tiếng việt để trả lời:
    Nhà viết kịch: Đây là bản tóm tắt của vở kịch trên:"""
)

review_prompt = PromptTemplate.from_template(
    """Bạn là nhà phê bình vở kịch của tờ New York Times. Đưa ra bản tóm tắt của vở kịch, công việc của bạn là viết bình luận cho vở kịch đó.
    Lưu ý: Bạn chỉ được sử dụng tiếng việt để trả lời:
    Play Synopsis:
    {synopsis}
    Đánh giá từ một nhà phê bình vở kịch trên tờ New York Times về vở kịch trên:"""
)

llm = ChatGroq(
    model="llama3-70b-8192",
    api_key= userdata.get("GROQ_API_KEY")
)

chain = (
    {"synopsis": synopsis_prompt | llm | StrOutputParser()}
    | review_prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke({"title": "Bi kịch lúc hoàng hôn trên bãi biển"})

print(response)

**Đánh giá từ một nhà phê bình vở kịch trên tờ New York Times**

"Bi kịch lúc hoàng hôn trên bãi biển" là một vở kịch đầy ắp bí mật, thù hận và tai họa, đã mang đến cho khán giả một bức tranh về gia đình giàu có nhưng lại đầy vết thương. Nội dung của vở kịch xoay quanh nhân vật chính Linh, người quay trở lại quê hương sau nhiều năm đi biển để tham dự lễ kỷ niệm 50 năm ngày cưới của cha mẹ. Tuy nhiên, buổi lễ này lại trở thành một dịp để các bí mật và thù hận được lộ dần, đẩy gia đình này vào vòng xoáy của tai họa và bi kịch.

Diễn xuất của các diễn viên trong vở kịch này là tuyệt vời, đặc biệt là diễn viên thủ vai Linh và Nhi. Họ đã mang đến cho khán giả một bức tranh sống động về hai nhân vật này, với những cảm xúc và xung đột nội tâm được thể hiện một cách chân thực.

Vở kịch này cũng đã đặt ra nhiều câu hỏi về gia đình, về tình yêu, về thù hận và về sự trả thù. Nó là một câu chuyện về con người, về những bí mật và thù hận được che giấu đằng sau vẻ ngoài hào hoa của một gia đình giàu